In [1]:
import pandas as pd

In [10]:
df1_10 = pd.read_excel('1_tr_phrase_countvectorizer.xlsx', sheet_name='1_tr_phrase_countvectorizer_10')
df1_5 = pd.read_excel('1_tr_phrase_countvectorizer.xlsx', sheet_name='1_tr_phrase_countvectorizer_5')
df1_3 = pd.read_excel('1_tr_phrase_countvectorizer.xlsx', sheet_name='1_tr_phrase_countvectorizer_3')

df2_10 = pd.read_excel('2_tr_phrase_pos_pattern.xlsx', sheet_name='2_tr_phrase_pos_pattern_10')
df2_5 = pd.read_excel('2_tr_phrase_pos_pattern.xlsx', sheet_name='2_tr_phrase_pos_pattern_5')
df2_3 = pd.read_excel('2_tr_phrase_pos_pattern.xlsx', sheet_name='2_tr_phrase_pos_pattern_3')

df3_10 = pd.read_excel('3_tr_posfilter.xlsx', sheet_name='3_tr_posfilter_10')
df3_5 = pd.read_excel('3_tr_posfilter.xlsx', sheet_name='3_tr_posfilter_5')
df3_3 = pd.read_excel('3_tr_posfilter.xlsx', sheet_name='3_tr_posfilter_3')


df4_10 = pd.read_excel('4_tr_posfilter_posisi.xlsx', sheet_name='4_tr_posfilter_posisi_10')
df4_5 = pd.read_excel('4_tr_posfilter_posisi.xlsx', sheet_name='4_tr_posfilter_posisi_5')
df4_3 = pd.read_excel('4_tr_posfilter_posisi.xlsx', sheet_name='4_tr_posfilter_posisi_3')

df5_10 = pd.read_excel('5_tr_weightscorephrases.xlsx', sheet_name='5_tr_weightscorephrases_10')
df5_5 = pd.read_excel('5_tr_weightscorephrases.xlsx', sheet_name='5_tr_weightscorephrases_5')
df5_3 = pd.read_excel('5_tr_weightscorephrases.xlsx', sheet_name='5_tr_weightscorephrases_3')

df6_10 = pd.read_excel('6_tr_tfidfscore.xlsx', sheet_name='6_tr_tfidfscore_10')
df6_5 = pd.read_excel('6_tr_tfidfscore.xlsx', sheet_name='6_tr_tfidfscore_5')
df6_3 = pd.read_excel('6_tr_tfidfscore.xlsx', sheet_name='6_tr_tfidfscore_3')

In [19]:
import pandas as pd

# Helper function to calculate F1 score
def calculate_f1(precision, recall):
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

# List of files and sheets to process
'''
files_and_sheets = {
    '1_tr_phrase_countvectorizer.xlsx': [
        '1_tr_phrase_countvectorizer_10',
        '1_tr_phrase_countvectorizer_5',
        '1_tr_phrase_countvectorizer_3'
    ],
    '2_tr_phrase_pos_pattern.xlsx': [
        '2_tr_phrase_pos_pattern_10',
        '2_tr_phrase_pos_pattern_5',
        '2_tr_phrase_pos_pattern_3'
    ],
    '3_tr_posfilter.xlsx': [
        '3_tr_posfilter_10',
        '3_tr_posfilter_5',
        '3_tr_posfilter_3'
    ],
    '4_tr_posfilter_posisi.xlsx': [
        '4_tr_posfilter_posisi_10',
        '4_tr_posfilter_posisi_5',
        '4_tr_posfilter_posisi_3'
    ],
    '5_tr_weightscorephrases.xlsx': [
        '5_tr_weightscorephrases_10',
        '5_tr_weightscorephrases_5',
        '5_tr_weightscorephrases_3'
    ],
    '6_tr_tfidfscore.xlsx': [
        '6_tr_tfidfscore_10',
        '6_tr_tfidfscore_5',
        '6_tr_tfidfscore_3'
    ]# Add more files and sheets as needed
}
'''
# List of files to process
files = [
    '1_tr_phrase_countvectorizer.xlsx',
    '2_tr_phrase_pos_pattern.xlsx',
    '3_tr_posfilter.xlsx',
    '4_tr_posfilter_posisi.xlsx',
    '5_tr_weightscorephrases.xlsx',
    '6_tr_tfidfscore.xlsx'
]


# Creating a list to store the results
results = []

# Loop through each file, calculate means and F1 scores, and store them in the results list
for file in files:
    result = {'Excel File': file}

    # Fixed sheet names based on the provided column naming pattern
    sheets = ['10', '5', '3']
    
    for sheet_suffix in sheets:
        sheet_name = f"{file.split('.')[0]}_{sheet_suffix}"
        df = pd.read_excel(file, sheet_name=sheet_name)

        mean_flex_recall = df['flex_recall'].mean()
        mean_flex_prec = df['flex_prec'].mean()
        mean_f1_score = calculate_f1(mean_flex_prec, mean_flex_recall)
        
        prefix = f"top-{sheet_suffix}"
        result[f'{prefix} + flex_recall'] = mean_flex_recall
        result[f'{prefix} + flex_prec'] = mean_flex_prec
        result[f'{prefix} + flex_F1'] = mean_f1_score

    results.append(result)

# Creating a DataFrame to display the results
result_df = pd.DataFrame(results)
print(result_df)

# Save the result DataFrame to an Excel file
result_df.to_excel('results.xlsx', index=False)
print("Results have been saved to 'results.xlsx'.")

                         Excel File  top-10 + flex_recall  top-10 + flex_prec  \
0  1_tr_phrase_countvectorizer.xlsx              0.317515              0.2222   
1      2_tr_phrase_pos_pattern.xlsx              0.190419              0.1332   
2               3_tr_posfilter.xlsx              0.180860              0.1265   
3        4_tr_posfilter_posisi.xlsx              0.280425              0.1962   
4      5_tr_weightscorephrases.xlsx              0.262995              0.1840   
5              6_tr_tfidfscore.xlsx              0.263706              0.1845   

   top-10 + flex_F1  top-5 + flex_recall  top-5 + flex_prec  top-5 + flex_F1  \
0          0.261441             0.189272             0.2648         0.220755   
1          0.156751             0.098955             0.1384         0.115400   
2          0.148873             0.098241             0.1374         0.114567   
3          0.230871             0.201867             0.2824         0.235437   
4          0.216517             

In [20]:
# Save the result DataFrame to an Excel file
result_df.to_excel('summary_results.xlsx', index=False)
print("Results have been saved to 'results.xlsx'.")

Results have been saved to 'results.xlsx'.
